In [4]:
from collections import defaultdict
import re

#  إنشاء ملفات تجريبية
# data.txt - Word Count
with open("data.txt", "w") as f:
    f.write("""spark makes big data easy
big data needs spark power
python is great for data processing""")

# sales.txt - Sales per Region
with open("sales.txt", "w") as f:
    f.write("""Product,Quantity,UnitPrice,Region
Apple,3,2.5,North
Banana,6,1.2,South
Apple,1,2.5,East
Orange,4,1.8,North
Banana,2,1.2,South
Apple,5,2.5,West
Orange,3,1.8,East
Apple,4,2.6,North
Banana,8,1.1,East
Apple,7,2.5,South
Orange,5,1.9,West
Banana,3,1.3,North
Grapes,10,3.2,East
Grapes,5,3.2,South
Banana,9,1.2,West
Apple,6,2.4,East
Orange,4,1.8,South
Grapes,8,3.3,North
Apple,2,2.5,North
Banana,5,1.2,East
Grapes,12,3.1,West
Apple,9,2.5,West
Orange,7,1.7,South
Grapes,6,3.2,East
Banana,11,1.1,South
Apple,10,2.6,North
Orange,2,1.9,West
Grapes,9,3.0,North
Apple,3,2.4,East
Banana,7,1.3,West
Apple,8,2.5,South
Orange,5,1.8,North
Grapes,10,3.2,West
Apple,4,2.5,East
Banana,6,1.2,North
Orange,9,1.9,East
Apple,7,2.6,South
Grapes,5,3.3,West
Banana,12,1.1,South
Apple,2,2.5,North
Orange,8,1.8,West
Grapes,7,3.1,East
Apple,11,2.4,East
Banana,5,1.2,West
Orange,3,1.9,North
Grapes,4,3.2,South
Apple,9,2.6,North
Banana,10,1.2,East
Grapes,8,3.1,West
Apple,5,2.5,South
Orange,7,1.7,East
Grapes,6,3.0,North
Banana,8,1.2,South
Apple,4,2.4,West
Orange,9,1.8,North
Grapes,5,3.3,East
Banana,11,1.1,West
Apple,3,2.5,South
Orange,2,1.9,South
Grapes,9,3.0,East
Apple,8,2.6,North
Banana,6,1.2,East
Orange,4,1.8,West
Grapes,10,3.2,South
Apple,12,2.4,East
Banana,9,1.1,North
Orange,3,1.9,East
Grapes,11,3.1,West
Apple,7,2.5,South
Banana,5,1.3,West
Orange,6,1.8,North
Grapes,8,3.2,East""")

# weblogs.txt - HTTP Logs
with open("weblogs.txt", "w") as f:
    f.write("""# Date, Time, IP, Method, URL, Status, ResponseSize
2025-10-10,12:01:32,192.168.1.2,GET,/index.html,200,1024
2025-10-10,12:01:33,192.168.1.3,GET,/products.html,200,850
2025-10-10,12:01:35,192.168.1.4,GET,/contact.html,404,512
2025-10-10,12:01:38,192.168.1.5,POST,/checkout,500,128
2025-10-10,12:01:41,192.168.1.6,GET,/index.html,200,1024
2025-10-10,12:01:45,192.168.1.7,GET,/images/logo.png,200,256
2025-10-10,12:01:48,192.168.1.8,GET,/about.html,404,512
2025-10-10,12:01:53,192.168.1.9,POST,/login,403,64
2025-10-10,12:02:01,192.168.1.10,GET,/index.html,200,1024
2025-10-10,12:02:07,192.168.1.11,POST,/checkout,500,128
2025-10-10,12:02:12,192.168.1.12,GET,/contact.html,404,512
2025-10-10,12:02:15,192.168.1.13,GET,/index.html,200,1024
2025-10-10,12:02:21,192.168.1.14,GET,/products.html,200,850
2025-10-10,12:02:23,192.168.1.15,GET,/about.html,404,512
2025-10-10,12:02:29,192.168.1.16,POST,/checkout,500,128
2025-10-10,12:02:31,192.168.1.17,GET,/images/logo.png,200,256
2025-10-10,12:02:34,192.168.1.18,GET,/contact.html,404,512
2025-10-10,12:02:38,192.168.1.19,POST,/login,403,64
2025-10-10,12:02:41,192.168.1.20,GET,/index.html,200,1024
2025-10-10,12:02:47,192.168.1.21,GET,/products.html,200,850""")

#  Word Count
def mapper_wordcount(line):
    words = re.findall(r"\b\w+\b", line.lower())
    return [(w, 1) for w in words]

def group_pairs(pairs):
    grouped = defaultdict(list)
    for k, v in pairs:
        grouped[k].append(v)
    return grouped

def reducer_sum(grouped):
    return {k: sum(v) for k, v in grouped.items()}

# Run Word Count
pairs = []
with open("data.txt", "r") as f:
    for line in f:
        pairs.extend(mapper_wordcount(line))
grouped = group_pairs(pairs)
result_wordcount = reducer_sum(grouped)

print("Word Count Results")
for word, count in sorted(result_wordcount.items(), key=lambda x: -x[1]):
    print(f"{word}: {count}")

#  Sales per Region
import csv

def mapper_sales(row):
    qty = float(row['Quantity'])
    price = float(row['UnitPrice'])
    region = row['Region']
    revenue = qty * price
    return [(region, revenue)]

pairs = []
with open("sales.txt", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        pairs.extend(mapper_sales(row))

grouped = group_pairs(pairs)
result_sales_region = reducer_sum(grouped)

print("\n Total Revenue per Region")
for region, total in result_sales_region.items():
    print(f"{region} → {total:.2f}")

#  Bonus: Total Revenue per Product

def mapper_sales_product(row):
    qty = float(row['Quantity'])
    price = float(row['UnitPrice'])
    product = row['Product']
    revenue = qty * price
    return [(product, revenue)]

pairs = []
with open("sales.txt", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        pairs.extend(mapper_sales_product(row))

grouped = group_pairs(pairs)
result_sales_product = reducer_sum(grouped)

print("\n Total Revenue per Product")
for product, total in result_sales_product.items():
    print(f"{product} → {total:.2f}")

#  Log Analysis

def mapper_status(line):
    parts = line.strip().split(",")
    if len(parts) < 6: return []
    status = parts[5].strip()
    return [(status, 1)] if status else []

def mapper_url(line):
    parts = line.strip().split(",")
    if len(parts) < 5: return []
    url = parts[4].strip()
    return [(url, 1)] if url else []

def mapper_status_size(line):
    parts = line.strip().split(",")
    if len(parts) < 7: return []
    status = parts[5].strip()
    try:
        size = int(parts[6].strip())
    except:
        size = 0
    return [(status, size)]

# Status Count
pairs = []
with open("weblogs.txt", "r") as f:
    for line in f:
        pairs.extend(mapper_status(line))
grouped = group_pairs(pairs)
status_counts = reducer_sum(grouped)

# URL Count
pairs = []
with open("weblogs.txt", "r") as f:
    for line in f:
        pairs.extend(mapper_url(line))
grouped = group_pairs(pairs)
url_counts = reducer_sum(grouped)

# Status Response Size
pairs = []
with open("weblogs.txt", "r") as f:
    for line in f:
        pairs.extend(mapper_status_size(line))
grouped = group_pairs(pairs)
status_sizes = reducer_sum(grouped)

# Display Log Results
print("\n Requests per Status Code ")
for status, count in sorted(status_counts.items()):
    print(f"HTTP {status}: {count} requests")

print("\n Requests per URL")
for url, count in sorted(url_counts.items()):
    print(f"{url}: {count} requests")

print("\n Total Response Size per Status ")
for status, total_size in sorted(status_sizes.items()):
    print(f"HTTP {status}: {total_size} bytes")

print("\n Errors Only (Status != 200)")
for status, count in sorted(status_counts.items()):
    if status != '200':
        print(f"HTTP {status}: {count} requests")


Word Count Results
data: 3
spark: 2
big: 2
makes: 1
easy: 1
needs: 1
power: 1
python: 1
is: 1
great: 1
for: 1
processing: 1

 Total Revenue per Region
North → 239.40
South → 221.40
East → 305.40
West → 268.60

 Total Revenue per Product
Apple → 325.20
Banana → 144.00
Orange → 146.80
Grapes → 418.80

 Requests per Status Code 
HTTP 200: 10 requests
HTTP 403: 2 requests
HTTP 404: 5 requests
HTTP 500: 3 requests
HTTP Status: 1 requests

 Requests per URL
/about.html: 2 requests
/checkout: 3 requests
/contact.html: 3 requests
/images/logo.png: 2 requests
/index.html: 5 requests
/login: 2 requests
/products.html: 3 requests
URL: 1 requests

 Total Response Size per Status 
HTTP 200: 8182 bytes
HTTP 403: 128 bytes
HTTP 404: 2560 bytes
HTTP 500: 384 bytes
HTTP Status: 0 bytes

 Errors Only (Status != 200)
HTTP 403: 2 requests
HTTP 404: 5 requests
HTTP 500: 3 requests
HTTP Status: 1 requests
